In [ ]:
# Filter out and put the corrupted record in new column

df = spark.read \
    .option("mode", "PERMISSIVE") \
    .option("columnNameOfCorruptRecord", "_corrupt_record") \
    .json("path/to/file.json")

df.select("_corrupt_record").where("_corrupt_record".isNotNull()).show(truncate=False)

In [2]:
from pyspark.sql import SparkSession
import datetime
spark = SparkSession.builder \
    .appName("Files Stream")\
    .config("spark.drive.host","127.0.0.1") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .config("spark.executor.memory","4g") \
    .config("spark.driver.memory","8g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark.conf.set("spark.sql.streaming.schemaInference",True)
# spark.sparkContext.setLogLevel("DEBUG")
print(f'spark session created {spark} at {datetime.datetime.now()}')
# spark.stop()
spark

spark session created <pyspark.sql.session.SparkSession object at 0x724cb03138c0> at 2025-06-22 22:15:04.930097


In [7]:
'''For reading streaning source'''
df=(spark.readStream.format("json")
    .option('multiLine',True)
    .load('devices/'))

In [4]:
df=(spark.read.format("json").option('multiLine',True)
    .load('devices/'))
# df.show()
# df.printSchema()

In [8]:
all_df=(df.selectExpr("customerId","eventId","eventOffset","eventPublisher","eventOffset","eventTime"
                      ,"explode(data.devices) as devices" )
).selectExpr("customerId","eventId","eventOffset","eventPublisher","eventOffset","eventTime"
             ,"devices.deviceId","devices.temperature","devices.measure","devices.status")
# all_df.show(truncate=False)
all_df.printSchema()

root
 |-- customerId: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- deviceId: string (nullable = true)
 |-- temperature: long (nullable = true)
 |-- measure: string (nullable = true)
 |-- status: string (nullable = true)



In [ ]:
all_df.writeStream.format("console").outputMode("append").start().awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+----------+--------------------+-----------+--------------+-----------+--------------------+--------+-----------+-------+-------+
|customerId|             eventId|eventOffset|eventPublisher|eventOffset|           eventTime|deviceId|temperature|measure| status|
+----------+--------------------+-----------+--------------+-----------+--------------------+--------+-----------+-------+-------+
|   CI00103|e3cb26d3-41b2-49a...|      10001|        device|      10001|2023-01-05 11:13:...|    D001|         15|      C|  ERROR|
|   CI00103|e3cb26d3-41b2-49a...|      10001|        device|      10001|2023-01-05 11:13:...|    D002|         16|      C|SUCCESS|
+----------+--------------------+-----------+--------------+-----------+--------------------+--------+-----------+-------+-------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/jarvis/.venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jarvis/.venv/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

25/06/23 07:23:08 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE